In [1]:
%pip install -qq llama_index=="0.8.2" pydantic=="1.10.12" nltk
# https://gpt-index.readthedocs.io/en/stable/core_modules/supporting_modules/evaluation/usage_pattern.html

Note: you may need to restart the kernel to use updated packages.


In [2]:
from llama_index import (
    VectorStoreIndex,
    SimpleWebPageReader,
    ServiceContext,
    LLMPredictor,
)
from llama_index.llms import OpenAI

urls = ["http://paulgraham.com/greatwork.html"]
documents = SimpleWebPageReader(html_to_text=True).load_data(urls)

llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="gpt-4"))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

index = VectorStoreIndex.from_documents(
    documents=documents, service_context=service_context
)
query_engine = index.as_query_engine()

In [3]:
from llama_index.evaluation import ResponseEvaluator

# https://gpt-index.readthedocs.io/en/v0.6.33/how_to/evaluation/evaluation.html#binary-evaluation
evaluator = ResponseEvaluator(service_context=service_context)
response = query_engine.query("How to do great work?")

eval_result1 = evaluator.evaluate(response)
eval_result2 = evaluator.evaluate_source_nodes(response)
print(str(eval_result1), str(eval_result2))

YES ['YES', 'NO']


In [ ]:
from llama_index.evaluation import QueryResponseEvaluator

evaluator = QueryResponseEvaluator(service_context=service_context)

query = "How to do great work?"
response = query_engine.query(query)

eval_result1 = evaluator.evaluate(query, response)
eval_result2 = evaluator.evaluate_source_nodes(query, response)
print(str(eval_result1), str(eval_result2))

YES ['YES', 'YES']


In [ ]:
# Question generation (Can be paired with eval above)

from llama_index.evaluation import DatasetGenerator

data_generator = DatasetGenerator.from_documents(documents)
eval_questions = data_generator.generate_questions_from_nodes(num=5)

for q in eval_questions:
    print(q)

What are the three qualities that the work you choose needs to have according to the author?
How does the author suggest figuring out what to work on?
What are the four steps the author outlines for doing great work?
Why does the author emphasize the importance of working on something you're deeply interested in?
How does the author suggest making yourself a big target for luck?
